# Normative DTI values in the pediatric spinal cord

This jupyter notebook includes scripts to generate figures related to normative DTI values in the pediatric spinal cord.

In [19]:
import sys, os
import argparse
import yaml
import pandas as pd
import re

Load data path

In [23]:
# Load config file to get paths to data
with open('../../config/config_preprocessing.yaml' , 'r') as file:
    config = yaml.safe_load(file)

# Get data path from config file
path_data = config['path_data']

In [24]:
# Get path to participants.tsv file
participants_tsv = pd.read_csv(os.path.join(path_data, 'participants.tsv'), sep='\t')
participants_tsv

,participant_id,age,sex,group,scan_series
0,sub-101,17,M,control,complete
1,sub-102,15,F,control,complete
2,sub-103,15,M,control,complete
3,sub-104,15,F,control,complete
4,sub-105,13,M,control,complete
...,...,...,...,...,...
110,sub-214,6,M,control,complete
111,sub-215,16,F,control,complete
112,sub-216,15,F,control,complete
113,sub-217,15,M,control,complete


In [28]:
# Get list of subjects to exclude from the analysis from the `exclude.yml` file (under the 'dwi' key)
with open(os.path.join(path_data, 'exclude.yml'), 'r') as file:
    exclude_yml = yaml.safe_load(file)

exclude_dwi_key = exclude_yml.get('dwi', []) # Extract subjects under 'dwi' key
exclude_subjects = sorted(set(re.match(r"(sub-\d+)", entry).group(1) for entry in exclude_dwi_key if re.match(r"(sub-\d+)", entry))) # Extract the subject ID 
print(exclude_subjects)

['sub-110', 'sub-117', 'sub-121', 'sub-129', 'sub-137', 'sub-138', 'sub-141', 'sub-144', 'sub-146', 'sub-150', 'sub-154', 'sub-159', 'sub-160', 'sub-164', 'sub-168', 'sub-169', 'sub-170', 'sub-171', 'sub-172', 'sub-180', 'sub-181', 'sub-186', 'sub-189', 'sub-190', 'sub-191', 'sub-193', 'sub-194', 'sub-196', 'sub-198', 'sub-199', 'sub-201', 'sub-203', 'sub-204', 'sub-207', 'sub-208', 'sub-209', 'sub-211', 'sub-214', 'sub-218']


Get DTI metrics and subject info (age, sex) in a single dataframe

In [ ]:
# DTI metric folders
DTI_folder = "../../results/DTI_metrics/"
metrics = ['FA', 'MD', 'AD', 'RD']

DTI_df = {}

for metric in metrics:
    metric_folder = os.path.join(DTI_folder, metric)
    metric_dfs = []
    
    for filename in os.listdir(metric_folder):
        if filename.endswith(".csv"):
            subject_path = os.path.join(metric_folder, filename)
            df = pd.read_csv(subject_path)
            subject_id = filename.split("_")[0]  # Get subject id from the filename (i.e., from 'sub-101_FA.csv')
            df["participant_id"] = subject_id
            metric_dfs.append(df)
    
    # Combine csv files of all subjects into a single dataframe for this metric
    DTI_df[metric] = pd.concat(metric_dfs, ignore_index=True)

# Add age and sex to DTI metric dataframe
for metric in DTI_df:
    DTI_df[metric] = DTI_df[metric].merge(participants_tsv, on="participant_id", how="left")


In [41]:
DTI_df['FA']

,Timestamp,SCT Version,Filename,Slice (I->S),VertLevel,DistancePMJ,Label,Size [vox],WA(),STD(),participant_id,age,sex,group,scan_series
0,2025-08-07 13:58:43,7.0,/home/samuelle/Documents/datasets/philadelphia...,5:7,5,NaN,WM right fasciculus gracilis,14.768168,0.674682,0.122756,sub-126,11,F,control,complete
1,2025-08-07 13:58:43,7.0,/home/samuelle/Documents/datasets/philadelphia...,8:9,4,NaN,WM right fasciculus gracilis,9.962411,0.680750,0.156440,sub-126,11,F,control,complete
2,2025-08-07 13:58:43,7.0,/home/samuelle/Documents/datasets/philadelphia...,10:12,3,NaN,WM right fasciculus gracilis,13.420370,0.733814,0.156204,sub-126,11,F,control,complete
3,2025-08-07 13:58:43,7.0,/home/samuelle/Documents/datasets/philadelphia...,13:14,2,NaN,WM right fasciculus gracilis,6.705773,0.795644,0.127063,sub-126,11,F,control,complete
4,2025-08-07 13:58:43,7.0,/home/samuelle/Documents/datasets/philadelphia...,5:7,5,NaN,WM left fasciculus cuneatus,16.803359,0.659559,0.152047,sub-126,11,F,control,complete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,2025-08-07 13:57:35,7.0,/home/samuelle/Documents/datasets/philadelphia...,8:10,4,NaN,lateral funiculi,93.116009,0.652078,0.135136,sub-106,14,M,control,complete
3128,2025-08-07 13:57:35,7.0,/home/samuelle/Documents/datasets/philadelphia...,11:14,3,NaN,lateral funiculi,125.490141,0.673214,0.148008,sub-106,14,M,control,complete
3129,2025-08-07 13:57:35,7.0,/home/samuelle/Documents/datasets/philadelphia...,6:7,5,NaN,ventral funiculi,30.891977,0.559655,0.131682,sub-106,14,M,control,complete
3130,2025-08-07 13:57:35,7.0,/home/samuelle/Documents/datasets/philadelphia...,8:10,4,NaN,ventral funiculi,41.707568,0.587378,0.144119,sub-106,14,M,control,complete


# Mean DTI values per age, per vertebral level

In [66]:
summary_tables = {}

for metric, df in DTI_df.items():
    # Optional: filter by spinal cord label if needed
    df_filtered = df[df['Label'] == 'spinal cord']  # or whatever label you care about

    # Group and compute mean
    grouped = df_filtered.groupby(['age', 'VertLevel'])['WA()'].mean().reset_index()
    grouped.rename(columns={'WA()': f'mean_{metric}'}, inplace=True)

    summary_tables[metric] = grouped

# Start with FA, then merge other metrics on 'VertLevel' and 'age'
final_summary = summary_tables['FA']
for metric in ['MD', 'AD', 'RD']:
    final_summary = final_summary.merge(summary_tables[metric], on=['VertLevel', 'age'], how='outer')

final_summary.to_csv("DTI_metrics_by_age_and_level.csv", index=False)
print(final_summary)

    age  VertLevel   mean_FA   mean_MD   mean_AD   mean_RD
0     6          2  0.681426  0.001024  0.001968  0.000552
1     7          2  0.624191  0.000981  0.001778  0.000582
2     9          2  0.628992  0.001158  0.002112  0.000681
3    10          2  0.657533  0.001014  0.001888  0.000576
4    11          2  0.668335  0.000968  0.001810  0.000547
5    12          2  0.646840  0.001017  0.001850  0.000600
6    13          2  0.645683  0.001009  0.001862  0.000583
7    14          2  0.679640  0.000960  0.001814  0.000533
8    15          2  0.695853  0.001009  0.001959  0.000534
9    16          2  0.662413  0.001001  0.001850  0.000577
10   17          2  0.667206  0.001014  0.001885  0.000579
11    6          3  0.606781  0.001131  0.001991  0.000701
12    7          3  0.621234  0.000994  0.001815  0.000583
13    9          3  0.509433  0.001237  0.001889  0.000911
14   10          3  0.611482  0.001079  0.001919  0.000660
15   11          3  0.655935  0.001002  0.001874  0.0005